# Trying different predicate sets

Naming things is a highly subjective process: different individuals will use different words to name their environment, and assess complexity of events differently, based on their subjective definitions. 

Our approach to memorability embraces this subjectivity in the definition of predicates and the evaluation of their complexity. In this example, we illustrate this phenomenon by considering the same memory of events, analyzed with different predicates.

In [ ]:
import sys
sys.path.append("../")

In [1]:
import pandas as pd
from abduction_memorability.event import Event, Label
from abduction_memorability.abduction_module import SurpriseAbductionModule
from abduction_memorability.predicate import *
from abduction_memorability.memory import Memory

import random
import math
from typing import Tuple

import plotly.io as pio
import plotly.express as px
import plotly.offline as py

## Step 1: Generating events

In [2]:
# defining labels
root_label = Label()
light_label = Label(name="light", parent=root_label)
blinds_label = Label(name="blinds", parent=root_label)
device_label = Label(name="device", parent=root_label)
device_addition_label = Label(name="addition", parent=device_label)
device_deletion_label = Label(name="deletion", parent=device_label)
tv_label = Label(name="tv_on", parent=root_label)


# some sensors
light_sensor = "light_sensor"
smart_tv = "smart_tv"
old_tv = "old_tv"
window = "window"
CURRENT_DAY = 100
EPOCH_SHIFT = 1577836800 # Epoch time of Jan, 1st 2021

# generating some events:
all_events: list[Event] = []
for day in range(0, CURRENT_DAY):
    for hour in range(0, 24):
        if day == CURRENT_DAY - 1 and hour == 13:
            event = Event(
                timestamp=day*86400 +  hour * 3600 + EPOCH_SHIFT, 
                duration=3600, 
                label=light_label,
                characteristics={
                    "luminosity": 100,
                    "device": light_sensor
                }
            )
            all_events.append(event)
            event = Event(
                timestamp=day*86400 + (hour-1) * 3600 + EPOCH_SHIFT,
                duration=3600,
                label=tv_label,
                characteristics={
                    "device": smart_tv
                }
            )
            all_events.append(event)
        else:
        # hour = 6 * hour
            if hour < 6 or hour > 18:
                event = Event(
                    timestamp=day*86400 +  hour * 3600 + EPOCH_SHIFT, 
                    duration=3600, 
                        label=light_label,
                        characteristics={
                            "luminosity": 100,
                            "device": light_sensor
                        }
                    )
                all_events.append(event)
            elif random.random() < 0.05:
                event = Event(
                    timestamp=day*86400 + hour * 3600 + EPOCH_SHIFT,
                    duration=3600,
                    label=light_label,
                    characteristics={
                        "luminosity": 200,
                        "device": light_sensor
                    }
                )
                all_events.append(event)
            else:
                if random.random() < 0.1 and day < CURRENT_DAY - 1:
                    event = Event(
                        timestamp=day*86400 + hour * 3600 + EPOCH_SHIFT,
                        duration=3600,
                        label=tv_label,
                        characteristics={
                            "device": old_tv
                        }
                    )
                    all_events.append(event)
                event = Event(
                    timestamp=day*86400 + hour * 3600 + EPOCH_SHIFT,
                    duration=3600,
                    label=light_label,
                    characteristics={
                        "luminosity": 1000,
                        "device": light_sensor
                    }
                )
                all_events.append(event)

len(all_events)

2519

In [3]:
memory = Memory(all_events)

## Step 2: Defining predicates

In [4]:

class ByDayPredicate(Predicate):
    def __init__(self, mem, prog, aux_predicate=None):
        super().__init__(mem, prog)
    
    def __call__(self, event):
        if self._prog > 0:
            return None
        mod_day = event.timestamp % 86400
        hour = mod_day / 3600
        if hour > 6 and hour <= 18:
            return True
        else:
            return False

    def __str__(self):
        return "day()"

class ByNightPredicate(Predicate):
    def __init__(self, mem, prog, aux_predicate=None):
        super().__init__(mem, prog)

    def __call__(self, event):
        if self._prog > 0:
            return None
        mod_day = event.timestamp % 86400
        hour = mod_day / 3600
        if hour <= 6 or hour > 18:
            return True
        else:
            return False

    def __str__(self):
        return "night()"

class Dark(Predicate):
    def __init__(self, mem, prog, aux_predicate=None):
        super().__init__(mem, prog, aux_predicate=aux_predicate)

    def __call__(self, event):
        if self._prog > 0:
            return None
        if event.get_char("luminosity") is None:
            return False
        return event.get_char("luminosity") < 300

    def __str__(self):
        return "dark()"

class Recent(Predicate):
    def __init__(self, mem, prog, aux_predicate=None):
        super().__init__(mem, prog, aux_predicate=aux_predicate)

    def __call__(self, event):
        if self.aux_predicate is None:
            if self._mem is None:
                last_date = CURRENT_DAY
            else:
                last_date = math.floor(self._mem.get_last_time() / 86400)
        else:
            last_date = math.floor(self.aux_predicate.timestamp / 86400)
            # print(last_date)
        if self._prog > 100:
            return None
        event_day = math.floor(event.timestamp / 86400)
        if last_date - event_day == self._prog:
            return True
        else:
            return False

    def __str__(self):
        return f"recent({self._prog})"

class Hour(Predicate):
    def __init__(self, mem, prog, aux_predicate=None):
        super().__init__(mem, prog, aux_predicate=aux_predicate)

    def __call__(self, event):
        if self._prog > 24:
            return None
        hour = event.timestamp % 86400
        hour = hour // 3600
        if self.aux_predicate is None:
            if hour == self._prog:
                return True
            return False
        else:
            other_event_hour = (self.aux_predicate.timestamp % 86400) // 3600
            diff_hour = min(abs(other_event_hour - hour), abs(other_event_hour - (24 + hour)))
            return diff_hour == self._prog

    def program_length(self):
        if self.aux_predicate is None:
            return math.log2(24) + 1
        else:
            return Helpers.bit_length(self._prog)                
    
    def __str__(self):
        return f'hour({self._prog})'
        

## Step 3: Analyzing

In [5]:
# The first module only knows predicates of time (day adn hour), alongside the label type
module1 = SurpriseAbductionModule(
    memory=memory,
    predicates=[
        DayPredicate,
        HasLabelPredicate,
        Hour
    ]
)

Loaded the memory with 2519 items!
Computing complexities with 4 passes
Starting pass 0 with 1 memories to explore
Finished pass 0 in 0.3387629985809326s.
Improved complexity for 0 event(s)
Starting pass 1 with 26 memories to explore
Finished pass 1 in 1.0350418090820312s.
Improved complexity for 1132 event(s)
Starting pass 2 with 200 memories to explore
Finished pass 2 in 1.4228804111480713s.
Improved complexity for 2765 event(s)
Computing surprise scores for all events !


In [6]:
df = module1.dataframe_output()

In [7]:
# Second module has knowledge of the notion of darkness and brightness, 
# alongside the notion of nigh/day

module2 = SurpriseAbductionModule(
    memory=memory,
    predicates=[
        DayPredicate,
        Hour,
        AxisRankPredicate,
        ByDayPredicate,
        Dark,
        ByNightPredicate,
        HasLabelPredicate
    ]
)

Loaded the memory with 2519 items!
Computing complexities with 4 passes
Starting pass 0 with 1 memories to explore
Finished pass 0 in 1.0927023887634277s.
Improved complexity for 0 event(s)
Starting pass 1 with 29 memories to explore
Finished pass 1 in 2.472660779953003s.
Improved complexity for 1132 event(s)
Starting pass 2 with 375 memories to explore
Finished pass 2 in 5.123936414718628s.
Improved complexity for 1402 event(s)
Starting pass 3 with 823 memories to explore
Finished pass 3 in 5.2664361000061035s.
Improved complexity for 0 event(s)
Computing surprise scores for all events !


In [8]:
df2 = module2.dataframe_output()

In [9]:
## Third module can select on particular event from the whole dataset, at the expense of 
# describing its address (i.e. using log(N) bits to describe any event from a memory of size N)
module3 = SurpriseAbductionModule(
    memory=memory,
    predicates=[
        RandomChoicePredicate,
        DayPredicate,
        Hour,
        AxisRankPredicate,
        ByDayPredicate,
        Dark,
        ByNightPredicate, 
        HasLabelPredicate
    ]
)
df3 = module3.dataframe_output()

Loaded the memory with 2519 items!
Computing complexities with 4 passes
Starting pass 0 with 1 memories to explore
Finished pass 0 in 1.5712106227874756s.
Improved complexity for 2519 event(s)
Starting pass 1 with 29 memories to explore
Finished pass 1 in 2.4260735511779785s.
Improved complexity for 215 event(s)
Starting pass 2 with 349 memories to explore
Finished pass 2 in 5.990077257156372s.
Improved complexity for 55 event(s)
Starting pass 3 with 217 memories to explore
Finished pass 3 in 4.0169150829315186s.
Improved complexity for 0 event(s)
Computing surprise scores for all events !


## Plotting the results

In [10]:
fig1 = px.scatter(df, x="date", y="complexity", color="label", hover_data=["id", "recipe"], color_discrete_sequence=["cadetblue", "lightsalmon"])
fig2 = px.scatter(df2, x="date", y="complexity", color="label", hover_data=["id", "recipe"], color_discrete_sequence=["cadetblue", "lightsalmon"])
fig3 = px.scatter(df3, x="date", y="complexity", color="label", hover_data=['id', 'recipe'], color_discrete_sequence=['cadetblue', 'lightsalmon'])

In [11]:
fig1.update_layout(
    showlegend=False,
    font=dict(
        size=30
    ),
    height=800,
    xaxis_visible=False
)
fig1

In [12]:
fig2.update_layout(
    showlegend=False,
    font=dict(
        size=30
    ),
    height=800,
    xaxis_visible=False
)
fig2

In [13]:
fig3.update_layout(
    showlegend=False,
    font=dict(
        size=30
    ),
    height=800
)
fig3

In [16]:
fig_mem = px.scatter(df2, x="date", y="memorability")

In [17]:
fig_mem